## Simulation in the Inventory Model ( Julia )

This code simulates the inventory of a firm many times.

It generates a matrix with `num_paths` rows and `sim_length` columns.

One simulation of inventory dynamics corresponds to one row.  

The question is: how well does naive parallelization work?

In [1]:
using Distributions

Threads.nthreads()

4

In [2]:
mutable struct SP
    s::Float64
    S::Float64
    p::Float64
end


function SP()
    
    s = 10.0
    S = 100.0
    p = 0.4
    SP(s, S, p)   
end

SP

## Sequential:

In [3]:
function sim_paths(sp::SP;initial_x=50.0, num_paths=100000,sim_length=400)
    
    X = Array{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    for i = 1:num_paths
        
        dvals = rand(Geometric(sp.p),sim_length)
        for t = 1:sim_length-1
            x, d = X[i, t], dvals[t]
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
            
            X[i, t+1] = y
            
            end
        end
       return X
    
end

sim_paths (generic function with 1 method)

In [4]:
sp = SP();

compilation time:

In [5]:
@time X = sim_paths(sp);

  2.367446 seconds (40.49 M allocations: 1.062 GiB, 9.70% gc time)


running time:

In [6]:
@time X = sim_paths(sp)

  2.238412 seconds (40.46 M allocations: 1.060 GiB, 7.33% gc time)


100000×400 Array{Int32,2}:
 50  50  49  48  47  46  45  44  44  44  …  59   56  56  56  56  55  55  55
 50  49  47  41  41  39  39  39  38  35     34   34  34  31  31  30  28  24
 50  49  48  46  46  43  43  43  42  39     99   99  99  99  99  99  98  97
 50  50  49  49  49  48  48  48  48  48     14   14  14  13  13  12  11  11
 50  50  45  45  43  41  41  41  41  41     76   75  73  73  69  68  68  67
 50  49  47  46  45  45  42  41  41  41  …  10   99  92  91  91  89  87  87
 50  50  50  50  50  48  46  46  46  40     97   96  96  93  93  87  84  84
 50  49  45  45  45  45  45  41  39  39     83   82  78  77  77  76  69  69
 50  50  48  48  47  47  47  47  47  46     90   86  86  86  82  81  79  77
 50  50  50  50  46  45  44  40  40  40     15   10  98  98  98  96  96  96
 50  48  48  45  45  42  41  38  38  31  …  45   43  42  42  37  37  35  34
 50  48  48  48  47  44  43  43  43  43     36   35  34  34  32  32  32  30
 50  48  41  41  40  40  39  34  34  34     63   63  61  60  

## Parallel:

In [7]:
function sim_paths(sp::SP;initial_x=50.0, 
                                num_paths=100000, sim_length=400)
    
    X = SharedArray{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    Threads.@threads for i=1:num_paths
        dvals = rand(Geometric(sp.p),sim_length)
        
        for t=1:sim_length-1
            x, d = X[i, t], dvals[t]
            
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
        
            X[i, t+1] = y
            
        end
    end
    
    return X  
end

sim_paths (generic function with 1 method)

compilation time:

In [8]:
@time X = sim_paths(sp);

  2.637535 seconds (23.78 M allocations: 593.046 MiB, 2.66% gc time)


running time:

In [9]:
@time X = sim_paths(sp)

  1.257406 seconds (22.57 M allocations: 528.922 MiB, 2.50% gc time)


100000×400 SharedArray{Int32,2}:
 50  49  45  45  45  43  42  39  39  …  55  52  51  50  50  50  45  40  33
 50  48  48  47  47  41  41  41  41     18  15  14  13  13  11  11  10  96
 50  50  47  45  45  45  41  41  41     98  97  95  95  95  93  90  90  89
 50  49  48  46  46  45  43  40  40     99  96  93  91  91  91  91  91  91
 50  46  44  41  40  40  40  40  40     39  38  37  27  25  22  16  16  16
 50  47  42  39  35  33  33  33  27  …  20  20  20  20  18  18  18  17  15
 50  48  46  46  46  46  46  46  45     16  16  16  16  16  16  15  13  11
 50  49  48  46  46  41  41  36  35     95  90  89  89  87  85  85  84  83
 50  50  50  50  49  48  45  41  39     14  14  13  12  11  10  96  96  94
 50  49  45  45  41  37  34  32  30     75  70  70  69  66  61  61  60  60
 50  50  49  49  49  49  49  46  46  …  49  49  49  47  38  37  34  34  33
 50  49  49  49  49  47  36  36  33     60  59  59  59  52  50  48  46  43
 50  50  49  48  48  46  44  43  43     92  83  80  78  75  74  74 